In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
import re
from fake_useragent import UserAgent

ua = UserAgent(use_external_data=True)
my_headers = {
    'user-agent': ua.random
}

folderPath = 'The Gutenberg Project'

num_of_books = 350

book_name_list = [] # 準備放每一本書的名字與第一層連結
story = [] # 書的第二層URL
story_content = [] # 書的第三層URL == 抓書中內文
regex_en = '^[a-zA-Z]' # 正規表達式挑出英文


def get_n_books():
    url = "https://www.gutenberg.org/browse/languages/zh" # 古騰堡網站第一層
    res = req.get(url, headers=my_headers) #GET 取得物件
    soup = bs(res.text, 'lxml') # res.text網站取得程式碼

    all_books_tags = soup.select('li.pgdbetext > a') # 每本書的a tag 抓出來

    count = 0
    for each_tag in all_books_tags:
        if re.search(regex_en, each_tag.get_text()) is None: # 如果每本書的書名不包含英文,就挑出來 (is None)==True
            book_name_list.append({ # book_name_list有了全中文的書名(理想上) & 第一層的URL
                "Name": each_tag.get_text(),
                "Url": "https://www.gutenberg.org/" + each_tag['href']
            }) 
            count+=1

            if count == num_of_books: # 我只抓num_of_books本書
                break
                
def get_books_url():
    for book in book_name_list:
        story.append({
            'Name': book['Name'],
            'Content_url': book['Url']+".html.noimages"
        })
        
def get_book_content():
    for article in story:  # story中的每一本書
        article_access = req.get(article['Content_url'], headers=my_headers)  # GET每本書
        article_soup = bs(article_access.text, 'lxml')  # 解析每本書的程式碼
        article_content = article_soup.select('body > p')  # 取得每一個p tag
        temp1 = [] # 暫存區1
        for p in article_content:
            if re.search(regex_en, p.get_text()) is None: # 若p不是英文開頭
                temp1.append(p.get_text())  # 將每一個段落p 內文丟到temp1中
        temp2 = "".join(temp1)  # 把temp1的所有分開的字串結合在一起
        story_content.append(temp2)  # temp2輸出到story_content中,完成一本書的內文記錄
        

def generate_book_txt():
    for i in range(num_of_books):
        try:
            with open(f"{folderPath}/{story[i]['Name']}.txt", "w", encoding = "utf-8") as file:
                file.write(story_content[i])
        except OSError:
            print(f"{story[i]['Name']} is failed.")
            continue
            
'''
主程式
'''

if __name__ == "__main__":
    get_n_books()
    get_books_url()
    get_book_content()
    generate_book_txt()